In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
from datetime import datetime
from glob import glob

In [23]:
# 1. Скачать список популярных производителей моделей
# 2. Скачать список популярных моделей
# 3. Сгенерировать ссылки на страницы моделей № 1-99

### 1. Генерируем ссылки на страницы поиска

In [2]:
# 1. Скачать список популярных производителей моделей
#catalog_ref = 'https://auto.ru/rossiya/cars/used/?sort=fresh_relevance_1-desc'
catalog_ref = 'https://auto.ru/rossiya/cars/used/?sort=fresh_relevance_1-desc'
response_catalog = requests.get(catalog_ref, verify=True)
html_catalog = response_catalog.content
soup_catalog = BeautifulSoup(html_catalog,'html.parser')

find_mask = 'Link ListingPopularMMM-module__itemName'
atags = soup_catalog.find_all('a',  attrs = {'class':find_mask})
producer_ref = [a.get('href') for a in atags]

In [3]:
producer_ref

['https://auto.ru/rossiya/cars/bmw/used/',
 'https://auto.ru/rossiya/cars/chevrolet/used/',
 'https://auto.ru/rossiya/cars/ford/used/',
 'https://auto.ru/rossiya/cars/hyundai/used/',
 'https://auto.ru/rossiya/cars/kia/used/',
 'https://auto.ru/rossiya/cars/vaz/used/',
 'https://auto.ru/rossiya/cars/mercedes/used/',
 'https://auto.ru/rossiya/cars/nissan/used/',
 'https://auto.ru/rossiya/cars/opel/used/',
 'https://auto.ru/rossiya/cars/toyota/used/',
 'https://auto.ru/rossiya/cars/volkswagen/used/']

In [4]:
# 2. Скачать список популярных моделей
# необходимо добавить проверку бана при автоматическом запросе
model_ref = []
offer_counts = []
for producer in producer_ref:
    print(producer)
    response_model = requests.get(producer, verify=True)
    html_model = response_model.content
    soup_model = BeautifulSoup(html_model,'html.parser')
        
    find_model_mask = 'Link ListingPopularMMM-module__itemName'
    atags = soup_model.find_all('a',  attrs = {'class':find_model_mask})
    value = 'ListingPopularMMM-module__itemCount'
    div_tags = soup_model.find_all('div',  attrs = {'class': value})
    model_ref += [a.get('href') for a in atags]
    offer_counts += [int(div.text) for div in div_tags]
    
    #time.sleep(15)
model_dict = dict(zip(model_ref, offer_counts))

https://auto.ru/rossiya/cars/bmw/used/
https://auto.ru/rossiya/cars/chevrolet/used/
https://auto.ru/rossiya/cars/ford/used/
https://auto.ru/rossiya/cars/hyundai/used/
https://auto.ru/rossiya/cars/kia/used/
https://auto.ru/rossiya/cars/vaz/used/
https://auto.ru/rossiya/cars/mercedes/used/
https://auto.ru/rossiya/cars/nissan/used/
https://auto.ru/rossiya/cars/opel/used/
https://auto.ru/rossiya/cars/toyota/used/
https://auto.ru/rossiya/cars/volkswagen/used/


Сохраним данные в датафрейм с укзанием метки времени, в дальнейшем по этим данным можно будет выполнить анализ динамики рынка подержанных авто в разрезе долей производителей и моделей, за временной период. Для этого наберем статистику за период времени с ежедневными замерами.

In [5]:
# create dataframe for models pages
model_frame = pd.DataFrame()
model_frame['producer'] = [m.split('/')[5] for m in model_ref]
model_frame['model'] = [m.split('/')[6] for m in model_ref]
model_frame['offer_counts'] = offer_counts
model_frame['href'] = model_ref
model_frame.to_csv('./time_offer_data/model_data_offer_used ' \
                   + datetime.today().strftime('%Y-%m-%d %H:%M') + '.csv')

In [6]:
model_frame.head()

,producer,model,offer_counts,href
0,bmw,1er,747,https://auto.ru/rossiya/cars/bmw/1er/used/
1,bmw,3er,3243,https://auto.ru/rossiya/cars/bmw/3er/used/
2,bmw,5er,4545,https://auto.ru/rossiya/cars/bmw/5er/used/
3,bmw,6er,325,https://auto.ru/rossiya/cars/bmw/6er/used/
4,bmw,7er,1248,https://auto.ru/rossiya/cars/bmw/7er/used/


In [18]:
model_frame.head()

,producer,model,offer_counts,href
0,bmw,1er,1046,https://auto.ru/rossiya/cars/bmw/1er/used/?sor...
1,bmw,3er,4093,https://auto.ru/rossiya/cars/bmw/3er/used/?sor...
2,bmw,4,184,https://auto.ru/rossiya/cars/bmw/4/used/?sort=...
3,bmw,5er,5291,https://auto.ru/rossiya/cars/bmw/5er/used/?sor...
4,bmw,6er,361,https://auto.ru/rossiya/cars/bmw/6er/used/?sor...


In [20]:
# 3. Сгенерировать ссылки на страницы моделей № 1-99
#addon = '&output_type=table&page='
addon = '&output_type=list&page='
search_pages = []
for mask_page in model_ref:
    page_count = 99  if int(np.ceil(model_dict[mask_page]/37)) > 99 else int(np.ceil(model_dict[mask_page]/37))
    for i in range(1, page_count+1):
        search_pages.append(mask_page.replace('?sort=fresh_relevance_1-desc', '?sort=alphabet-asc')
                            + addon + str(i))

In [21]:
search_pages[:5]

['https://auto.ru/rossiya/cars/bmw/1er/used/?sort=alphabet-asc&output_type=list&page=1',
 'https://auto.ru/rossiya/cars/bmw/1er/used/?sort=alphabet-asc&output_type=list&page=2',
 'https://auto.ru/rossiya/cars/bmw/1er/used/?sort=alphabet-asc&output_type=list&page=3',
 'https://auto.ru/rossiya/cars/bmw/1er/used/?sort=alphabet-asc&output_type=list&page=4',
 'https://auto.ru/rossiya/cars/bmw/1er/used/?sort=alphabet-asc&output_type=list&page=5']

In [391]:
page_df =pd.DataFrame()
page_df['page'] = search_pages
page_df.to_csv('page_'+datetime.today().strftime('%Y-%m-%d %H:%M')+'.csv')

In [22]:
# из-за ограничений отображения поиска доступны только ~73% объявлений
print(len(search_pages))
print(model_frame['offer_counts'].sum())
print(len(search_pages) * 37)
print(round(len(search_pages) * 37/model_frame['offer_counts'].sum(), 2))

6812
318599
252044
0.79


### 2. Собираем нужную информацию со страницы

In [408]:
%%time
# create list for dataframe collecting
total_frames = []
i = 0
# cycle around link list
while i < len(search_pages):
    
    try:
        # set page
        page = search_pages[i]

        # get html response
        response_search = requests.get(page, verify=True)
        html_search = response_search.content
        soup_search = BeautifulSoup(html_search,'html.parser')

        if response_search.status_code == 200:

            # extract data
            # technical tags
            tech_tags = [[el.text for el in t.find_all('div',  attrs = {'class':'ListingItemTechSummary-module__cell'})] \
                    for t in soup_search.find_all('div', \
                    attrs = {'class':'ListingItemTechSummary-module__container ListingItem-module__techSummary'})]
            # ref 
            refs = [t.get('href') for t in soup_search.find_all('a',  attrs = {'class':'Link ListingItemTitle-module__link'})]
            # name
            names = [t.text for t in soup_search.find_all('a',  attrs = {'class':'Link ListingItemTitle-module__link'})]
            # price
            prices = [t.text for t in soup_search.find_all('div',  attrs = {'class':'ListingItemPrice-module__content'})]
            # age ListingItem-module__year
            ages = [t.text for t in soup_search.find_all('div',  attrs = {'class':'ListingItem-module__year'})]

            # km
            milages = [t.text for t in soup_search.find_all('div',  attrs = {'class':'ListingItem-module__kmAge'})]

            # region
            regions = [t.text for t in soup_search.find_all('span',  attrs = {'class':'MetroListPlace__regionName MetroListPlace_nbsp'})]

            # images
            img_links = [[i.get('data-src') for i in img.find_all('div', attrs={'class':'LazyImage Brazzers__image'})] 
                                 for img in soup_search.find_all('div',  attrs = {'class':'Brazzers'})]

            # create temp dataframe
            temp = pd.DataFrame()

            temp['model_name'] = names
            temp['price'] = prices
            temp['age'] = ages
            temp['kmage'] = milages
            temp['region'] = regions
            temp['tech_dicript'] = tech_tags
            temp['offer_href'] = refs
            temp['img_links'] = img_links

            # append to offer list
            total_frames.append(temp)
            print('Appended page: ', i)
            # increment pointer if parse will be blocked we start with same link again
            i += 1

        else:
            print('Wrong link or bloked!', page)
            time.sleep(600)
            #break
            
    except Exception as e:
        print('Error!', page)
        print(e)
        break
        time.sleep(600)
        

Appended page:  1310
Appended page:  1311
Appended page:  1312
Appended page:  1313
Appended page:  1314
Appended page:  1315
Appended page:  1316
Appended page:  1317
Appended page:  1318
Appended page:  1319
Appended page:  1320
Appended page:  1321
Appended page:  1322
Appended page:  1323
Appended page:  1324
Appended page:  1325
Appended page:  1326
Appended page:  1327
Appended page:  1328
Appended page:  1329
Appended page:  1330
Appended page:  1331
Appended page:  1332
Appended page:  1333
Appended page:  1334
Appended page:  1335
Appended page:  1336
Appended page:  1337
Appended page:  1338
Appended page:  1339
Appended page:  1340
Appended page:  1341
Appended page:  1342
Appended page:  1343
Appended page:  1344
Appended page:  1345
Appended page:  1346
Appended page:  1347
Appended page:  1348
Appended page:  1349
Appended page:  1350
Appended page:  1351
Appended page:  1352
Appended page:  1353
Appended page:  1354
Appended page:  1355
Appended page:  1356
Appended page

Appended page:  1701
Appended page:  1702
Appended page:  1703
Appended page:  1704
Appended page:  1705
Appended page:  1706
Appended page:  1707
Appended page:  1708
Appended page:  1709
Appended page:  1710
Appended page:  1711
Appended page:  1712
Appended page:  1713
Appended page:  1714
Appended page:  1715
Appended page:  1716
Appended page:  1717
Appended page:  1718
Appended page:  1719
Appended page:  1720
Appended page:  1721
Appended page:  1722
Appended page:  1723
Appended page:  1724
Appended page:  1725
Appended page:  1726
Appended page:  1727
Appended page:  1728
Appended page:  1729
Appended page:  1730
Appended page:  1731
Appended page:  1732
Appended page:  1733
Appended page:  1734
Appended page:  1735
Appended page:  1736
Appended page:  1737
Appended page:  1738
Appended page:  1739
Appended page:  1740
Appended page:  1741
Appended page:  1742
Appended page:  1743
Appended page:  1744
Appended page:  1745
Appended page:  1746
Appended page:  1747
Appended page

Appended page:  2092
Appended page:  2093
Appended page:  2094
Appended page:  2095
Appended page:  2096
Appended page:  2097
Appended page:  2098
Appended page:  2099
Appended page:  2100
Appended page:  2101
Appended page:  2102
Appended page:  2103
Appended page:  2104
Appended page:  2105
Appended page:  2106
Appended page:  2107
Appended page:  2108
Appended page:  2109
Appended page:  2110
Appended page:  2111
Appended page:  2112
Appended page:  2113
Appended page:  2114
Appended page:  2115
Appended page:  2116
Appended page:  2117
Appended page:  2118
Appended page:  2119
Appended page:  2120
Appended page:  2121
Appended page:  2122
Appended page:  2123
Appended page:  2124
Appended page:  2125
Appended page:  2126
Appended page:  2127
Appended page:  2128
Appended page:  2129
Appended page:  2130
Appended page:  2131
Appended page:  2132
Appended page:  2133
Appended page:  2134
Appended page:  2135
Appended page:  2136
Appended page:  2137
Appended page:  2138
Appended page

Appended page:  2483
Appended page:  2484
Appended page:  2485
Appended page:  2486
Appended page:  2487
Appended page:  2488
Appended page:  2489
Appended page:  2490
Appended page:  2491
Appended page:  2492
Appended page:  2493
Appended page:  2494
Appended page:  2495
Appended page:  2496
Appended page:  2497
Appended page:  2498
Appended page:  2499
Appended page:  2500
Appended page:  2501
Appended page:  2502
Appended page:  2503
Appended page:  2504
Appended page:  2505
Appended page:  2506
Appended page:  2507
Appended page:  2508
Appended page:  2509
Appended page:  2510
Appended page:  2511
Appended page:  2512
Appended page:  2513
Appended page:  2514
Appended page:  2515
Appended page:  2516
Appended page:  2517
Appended page:  2518
Appended page:  2519
Appended page:  2520
Appended page:  2521
Appended page:  2522
Appended page:  2523
Appended page:  2524
Appended page:  2525
Appended page:  2526
Appended page:  2527
Appended page:  2528
Appended page:  2529
Appended page

Appended page:  2874
Appended page:  2875
Appended page:  2876
Appended page:  2877
Appended page:  2878
Appended page:  2879
Appended page:  2880
Appended page:  2881
Appended page:  2882
Appended page:  2883
Appended page:  2884
Appended page:  2885
Appended page:  2886
Appended page:  2887
Appended page:  2888
Appended page:  2889
Appended page:  2890
Appended page:  2891
Appended page:  2892
Appended page:  2893
Appended page:  2894
Appended page:  2895
Appended page:  2896
Appended page:  2897
Appended page:  2898
Appended page:  2899
Appended page:  2900
Appended page:  2901
Appended page:  2902
Appended page:  2903
Appended page:  2904
Appended page:  2905
Appended page:  2906
Appended page:  2907
Appended page:  2908
Appended page:  2909
Appended page:  2910
Appended page:  2911
Appended page:  2912
Appended page:  2913
Appended page:  2914
Appended page:  2915
Appended page:  2916
Appended page:  2917
Appended page:  2918
Appended page:  2919
Appended page:  2920
Appended page

KeyboardInterrupt: 

In [432]:
# concate all frames
#source_frame = pd.concat(total_frames)
source_frame_all.to_pickle('source_frame_all_offer' + datetime.today().strftime('%Y-%m-%d %H:%M') +'.pckl')

### 3. Формируем датасет

In [427]:
source_files = ['source_frame_0_337_2020-01-22 09:41.pckl',
                'source_frame_337_412_2020-01-22 09:53.pckl',
                'source_frame_413_1309_2020-01-22 11:00.pckl',
                'source_frame_1310_3000_2020-01-22 12:27.pckl',
                'source_frame_3000_5000_2020-01-22 12-17.pckl',
                'source_frame_5000_6702_2020-01-22 11-17.pckl']

frames = []
for file in source_files:
    frames.append(pd.read_pickle(file))
source_frame_all = pd.concat(frames)
source_frame_all.reset_index(drop=True, inplace=True)

In [435]:
source_frame_all.head(2)

,model_name,price,age,kmage,region,tech_dicript,offer_href,img_links
0,BMW 1 серия II (F20/F21) 116i,850 000 ₽,2012,81 000 км,Одинцово,"[1.6 л / 136 л.с. / Бензин, автомат, хэтчбек 3...",https://auto.ru/cars/used/sale/bmw/1er/1094869...,[//avatars.mds.yandex.net/get-autoru-vos/21601...
1,BMW 1 серия II (F20/F21) 116i,707 000 ₽,2013,68 880 км,Москва,"[1.6 л / 136 л.с. / Бензин, автомат, хэтчбек 3...",https://auto.ru/cars/used/sale/bmw/1er/1095356...,[//avatars.mds.yandex.net/get-autoru-vos/20737...


In [434]:
offer_frame = source_frame_all.copy(deep=True)

In [436]:
# clean data
offer_frame[['engine', 'gear', 'case', 'drive_type', 'color']] = \
                                offer_frame['tech_dicript'].apply(lambda row: pd.Series(row[:5]))
offer_frame[['engine_volume', 'engine_power', 'fuel']] = \
    offer_frame['engine'].apply(lambda row: pd.Series(row.replace('\u2009','').replace('\xa0',' ').split('/')))

offer_frame['price'] = offer_frame['price'].str.replace('\xa0','').str.replace('₽', '')
offer_frame['kmage'] = offer_frame['kmage'].str.replace('\xa0','').str.replace('км', '')
offer_frame['engine_volume'] = offer_frame['engine_volume'].str.replace(' л', '')
offer_frame['engine_power'] = offer_frame['engine_power'].str.replace(' л.с.', '')
offer_frame['producer'] = offer_frame['model_name'].str.split(' ').str.get(0)


# convert numeric data
int_cols = ['price', 'age', 'kmage', 'engine_volume', 'engine_power']
for col in int_cols:
    offer_frame[col] = pd.to_numeric(offer_frame[col], errors='coerce')

In [437]:
# subset_data
clean_offer_frame = offer_frame[['producer', 'model_name', 'price', 'age', 'kmage', 'region', 
                                 'engine_volume', 'engine_power', 'fuel','gear', 'case', 'drive_type', 
                                 'color',  'offer_href', 'img_links', 'tech_dicript']]
clean_offer_frame.to_pickle('clean_offer_frame_all' + datetime.today().strftime('%Y-%m-%d %H:%M') +'.pckl')

In [438]:
clean_offer_frame.head(2)

,producer,model_name,price,age,kmage,region,engine_volume,engine_power,fuel,gear,case,drive_type,color,offer_href,img_links,tech_dicript
0,BMW,BMW 1 серия II (F20/F21) 116i,850000,2012,81000,Одинцово,1.6,136.0,Бензин,автомат,хэтчбек 3 дв.,задний,белый,https://auto.ru/cars/used/sale/bmw/1er/1094869...,[//avatars.mds.yandex.net/get-autoru-vos/21601...,"[1.6 л / 136 л.с. / Бензин, автомат, хэтчбек 3..."
1,BMW,BMW 1 серия II (F20/F21) 116i,707000,2013,68880,Москва,1.6,136.0,Бензин,автомат,хэтчбек 3 дв.,задний,белый,https://auto.ru/cars/used/sale/bmw/1er/1095356...,[//avatars.mds.yandex.net/get-autoru-vos/20737...,"[1.6 л / 136 л.с. / Бензин, автомат, хэтчбек 3..."


In [439]:
clean_offer_frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 246324 entries, 0 to 246323
Data columns (total 16 columns):
producer         246324 non-null object
model_name       246324 non-null object
price            246324 non-null int64
age              246324 non-null int64
kmage            246324 non-null int64
region           246324 non-null object
engine_volume    246318 non-null float64
engine_power     246318 non-null float64
fuel             246324 non-null object
gear             246324 non-null object
case             246324 non-null object
drive_type       246324 non-null object
color            246324 non-null object
offer_href       246324 non-null object
img_links        246324 non-null object
tech_dicript     246324 non-null object
dtypes: float64(2), int64(3), object(11)
memory usage: 30.1+ MB


In [440]:
clean_offer_frame.iloc[0]

producer                                                       BMW
model_name                          BMW 1 серия  II (F20/F21) 116i
price                                                       850000
age                                                           2012
kmage                                                        81000
region                                                    Одинцово
engine_volume                                                  1.6
engine_power                                                   136
fuel                                                        Бензин
gear                                                       автомат
case                                                 хэтчбек 3 дв.
drive_type                                                  задний
color                                                        белый
offer_href       https://auto.ru/cars/used/sale/bmw/1er/1094869...
img_links        [//avatars.mds.yandex.net/get-autoru-vos/2160

## Выводы:
1. Создан код для сбора ежедневной статистики по 12 наиболее популярным производителями и их 132 моделям авто.
2. Собраны данные по 246k автомобилям выставленным на продажу, с описанием технических харакетиристик и внешнего вида, а также ссылки на фотографии данных транспортных средств, которые можно использовать для дальнешего анализа
3. Сформирован едлинный датасет который можно применить для дальнейшего анализа